In the comparison of algorithm , the SVM had an accuracy of 63.3%. In this case am considering svm only and specific variables and tuning some parameters. The conditions of fraud or not fraud still remain the same as explained in the project1 notebook.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import warnings
from sklearn import tree
from IPython.display import Image
import pydotplus as pydot 
warnings.filterwarnings("ignore")
from sklearn.externals.six import StringIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from numpy import set_printoptions
set_printoptions(precision=3)


In [7]:
Data = pd.read_excel("Gen.xlsx",sheet_name="data").dropna(how = "any")
Data = Data.dropna(how = 'any')

In [8]:
Data['consumption_perDay_within_a_period'] = Data['Consumption HIS']/(Data['NBRE DE JOUR'])
Data.loc[~np.isfinite(Data['consumption_perDay_within_a_period']), 'consumption_perDay_within_a_period'] = 0
Data['Quanitity_consumed_btn_visits'] = Data['QTE FUEL PRECEDENTE'] - Data['QTE FUEL TROUVEE']
set_printoptions(precision=3)

In [9]:
Data.drop(['Power Type'],axis=1, inplace=True)
Data.drop(['ANCIEN COMPTEUR HORAIRE G1'],axis=1, inplace=True)
Data.drop(['Class'],axis=1, inplace=True)
Data.drop(['DATE  EFFECTIVE DE LA VISITE'],axis=1, inplace=True)
Data.drop(['DATE DE VISITE PRECEDENTE'],axis=1, inplace=True)

Data.drop(['Months'],axis=1, inplace=True)
Data.drop(['Cluster'],axis=1, inplace=True)
Data.drop(['Site Name'],axis=1, inplace=True)
Data.drop(['Generator 1capacity (KVA)'],axis=1, inplace=True)

Data.drop(['COMPTEUR HORAIRE ACTUEL GE1'],axis=1, inplace=True)
Data.drop(['QTE FUEL AJOUTEE'],axis=1, inplace=True)


In [10]:

Data['Running time per day'] = Data["NBRE HEURES DE FONCTIONNEMENT DG1"]/Data["NBRE DE JOUR"]             
Data.loc[~np.isfinite(Data['Running time per day']), 'Running time per day'] = 0
set_printoptions(precision=3)

In [11]:
Data['Maximum_consumption_perDay'] = Data[['CONSOMMATION HORAIRE DE FUEL DG1']]*24
set_printoptions(precision=3)

maximum consumption in one day
fuel consumed per period difference netween the fuel found and previously left
comparison of the difference with running time.
Generator running for more than 24 hourse in a day


In [12]:
a = list(Data['NBRE HEURES DE FONCTIONNEMENT DG1'])
b = list(Data['Quanitity_consumed_btn_visits'])

In [63]:
c = []
for i in range(len(a)):
    if a[i] == 0  and  b[i]!=0:
        c.append(0)
    else:
        c.append(1)

In [14]:
t = list(Data['Maximum_consumption_perDay'])
r = list (Data['consumption_perDay_within_a_period'])

In [15]:
z = []
for i in range(len(r)):
    if r[i] > t[i]:
        z.append(0)
    else:
        z.append(1)

In [16]:
p = []
for i in range(len(z)):
    if c[i]==1 and z[i]==1:
        p.append(1)
    else:
        p.append(0)     

In [17]:
Data['Class'] = p


In [18]:
Data.drop(['QTE TOTALE LAISSEE'],axis=1, inplace=True)
#Data.drop(['Consumption HIS'],axis=1, inplace=True)
Data.drop(['QTE FUEL PRECEDENTE'],axis=1, inplace=True)
Data.drop(['QTE FUEL TROUVEE'],axis=1, inplace=True)
#Data.drop(['NBRE HEURES DE FONCTIONNEMENT DG1'],axis=1, inplace=True)
#Data.drop(['CONSOMMATION HORAIRE DE FUEL DG1'],axis=1, inplace=True)
Data.drop(['NBRE DE JOUR'],axis=1, inplace=True)
#Data.drop(['consumption_perDay_within_a_period'],axis=1, inplace=True)
Data.drop(['Maximum_consumption_perDay'],axis=1, inplace=True)
Data.drop(['Quanitity_consumed_btn_visits'],axis=1, inplace=True)

With five key variable that influence the consuption from the dataset. That is the total consumed in a period(Consumption HIS), the total number of hours the generator is running (NBRE HEURES DE FONCTIONNEMENT DG1), the rate at which the generator is consuming per hour (CONSOMMATION HORAIRE DE FUEL DG1) , the consumption per day(consumption_perDay_within_a_period) and   number of hours the generator is running per day(Running time per day).

In [20]:
Data.columns

Index(['Consumption HIS', 'NBRE HEURES DE FONCTIONNEMENT DG1',
       'CONSOMMATION HORAIRE DE FUEL DG1',
       'consumption_perDay_within_a_period', 'Running time per day', 'Class'],
      dtype='object')

In [26]:
x = Data.loc[:,Data.columns.tolist()[0:5]]
w = x.as_matrix()
Y = Data.loc[:,'Class']
Y = Y.as_matrix()

x_train,x_test,y_train,y_test = train_test_split(w,Y,test_size=0.20,random_state=0)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test) 
print(x.shape)
print(Y.shape)
print(x_test.shape)
print( x_train.shape)
print( y_test.shape)
print(y_train.shape)

(2676, 5)
(2676,)
(536, 5)
(2140, 5)
(536,)
(2140,)


In [22]:
Data.head(2)

,Consumption HIS,NBRE HEURES DE FONCTIONNEMENT DG1,CONSOMMATION HORAIRE DE FUEL DG1,consumption_perDay_within_a_period,Running time per day,Class
0,420.0,140.0,3.0,70.0,23.333333,1
1,254.8,98.0,2.6,63.7,24.500000,0


In [23]:
# FUNCTION FOR TRAINING THE MODEL
def trainmodel(model):
    model.fit(x_train,y_train)
# FUNCTION TO MAKE PREDICTIONS

def predictmodel(model):
    y_pred = model.predict(x_test)
    f,t,thresholds = metrics.roc_curve(y_test,y_pred)
    cm = metrics.confusion_matrix(y_test,y_pred)
    print("Score:", metrics.auc(f,t))
    print("Classification report:")
    print(metrics.classification_report(y_test,y_pred))
    print("Confusion Matrix:")
    print(cm)
def predictmodeln(model):
    y_pred = model.predict(x_test)
    f,t,thresholds = metrics.roc_curve(y_test,y_pred)
    cm = metrics.confusion_matrix(y_test,y_pred)
    print("Score:", metrics.auc(f,t))
    print("Classification report:")
    print(metrics.classification_report(y_test,y_pred))
    print("Confusion Matrix:")
    print(cm)

 Using the decision function ovo or ovr both have the same accuracy when  C and gamma are the same in all the case.
 With parameter C= 1 and gamma =0.1 we have a score 75%

In [62]:
SVM =SVC(kernel="rbf", C=1, gamma =0.1, decision_function_shape='ovo')
SVM.fit(x_train , y_train)
predictmodel(SVM)
SVM.predict(x_train)

Score: 0.7509630054834665
Classification report:
             precision    recall  f1-score   support

          0       0.86      0.55      0.67       193
          1       0.79      0.95      0.86       343

avg / total       0.81      0.81      0.79       536

Confusion Matrix:
[[107  86]
 [ 18 325]]


array([1, 1, 1, ..., 1, 1, 1])

With parameter C= 10 and gamma =0.1 we have a score 88%

In [48]:

SVM =SVC(kernel="rbf", C=10, gamma = 0.1, decision_function_shape='ovr')
SVM.fit(x_train , y_train)
predictmodel(SVM)

Score: 0.8840616927748154
Classification report:
             precision    recall  f1-score   support

          0       0.93      0.80      0.86       193
          1       0.90      0.97      0.93       343

avg / total       0.91      0.91      0.91       536

Confusion Matrix:
[[155  38]
 [ 12 331]]


With parameter C= 1 and gamma =1 we have a score 84%

In [49]:

SVM =SVC(kernel="rbf", C=1, gamma =1, decision_function_shape='ovr')
SVM.fit(x_train , y_train)
predictmodel(SVM)

Score: 0.843094306560522
Classification report:
             precision    recall  f1-score   support

          0       0.89      0.74      0.81       193
          1       0.86      0.95      0.91       343

avg / total       0.87      0.87      0.87       536

Confusion Matrix:
[[142  51]
 [ 17 326]]


With parameter C= 0.01 and gamma =1 we have a score 50%

In [52]:

SVM =SVC(kernel="rbf", C=0.01, gamma =1, decision_function_shape='ovr')
SVM.fit(x_train , y_train)
predictmodel(SVM)
SVM.predict(x_train)

Score: 0.5
Classification report:
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       193
          1       0.64      1.00      0.78       343

avg / total       0.41      0.64      0.50       536

Confusion Matrix:
[[  0 193]
 [  0 343]]


array([1, 1, 1, ..., 1, 1, 1])

Using the kford cross validation method to validate the performance of the algorithm , we have 63.35%

In [51]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
array = Data.values
X = array[:,0:5]
Y = array[:,5]
num_folds = 100
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)

results = cross_val_score(SVM, X, Y, cv=kfold)
results
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Accuracy: 63.353% (10.530%)
